In [341]:
import numpy as np
import pandas as pd

In [342]:
class NeuralNetwork:
    def __init__(self):
        #Status of each step, Add Layers must be fixed before Passing in Data
        self.status = False
        #A list of Hidden neurons, ith index is # of Neurons of ith layer
        self.hiddenNeurons = []
        #A list of Hidden Weights and Biases, ith index is Weight matrix to the next layer
        self.weights = []
        self.biases  = []
        #A list of dZ[i]
        self.dZ = []
        #A list of A[i], where A[0] is X(the inputs), A[n] is the outputs
        self.Z = []
        self.A = [] 
        #A list of Activation Functions
        self.activation_functions = []
        #Other
        self.y_train = None
        self.one_hot_Y = None
        self.n_in  = None #Number of Inputs
        self.n_out = None #Number of desired Outputs
        self.samples = None
        self.alpha = 0.1 #Adjustable
        self.epoch = 1000 #Adjustable

    def ReLU(self, Z):
        return np.maximum(0, Z)
    
    def ReLU_deriv(self, Z):
        return Z > 0
    
    def SoftMax(self, Z):
        A = np.exp(Z) / sum(np.exp(Z))
        return A
    
    def one_hot(self, Y):
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        return one_hot_Y.T
    
    def get_predictions(self, A):
        return np.argmax(A, 0)
    
    def get_accuracy(self, predictions, actual):
        passed = np.sum(predictions == actual)
        accuracy = passed / actual.size
        return accuracy
    
    def forward_propagation(self, i=0):
        if i == len(self.weights)-1:
            #Last Layers
            self.Z[i] = self.weights[i] @ self.A[i]
            self.A[i+1] = self.SoftMax(self.Z[i] + self.biases[i])
            return
        
        self.Z[i] = self.weights[i] @ self.A[i]
        self.A[i+1] = self.ReLU(self.Z[i] + self.biases[i])
        self.forward_propagation(i+1)

    def backward_propagation(self, i):
        dW = self.dZ[i] @ (self.A[i]).T
        self.weights[i] -= self.alpha * dW
        db = 1/self.samples * np.sum(self.dZ[i])
        self.biases[i]  -= self.alpha * db
        if i == 0:
            return
        self.dZ[i-1] = (self.weights[i]).T @ self.dZ[i] * self.ReLU_deriv(self.Z[i-1])
        self.backward_propagation(i-1) 

    def gradient_descent(self):
        self.one_hot_Y = self.one_hot(self.y_train)
        for i in range(self.epoch + 1):
            self.dZ[-1] = 1/self.samples * (self.A[-1] - self.one_hot_Y)
            self.forward_propagation()
            self.backward_propagation(len(self.dZ)-1)
            if i % 10 == 0:
                predictions = self.get_predictions(self.A[-1])
                accuracy = self.get_accuracy(predictions, self.y_train)
                print(f'Epoch {i}, accuracy={accuracy*100:.2f}%')

    #Generate Neural network Structure
    def addLayer(self, neurons, activation_function):
        if self.status: 
            return
        allowed = ['relu','softmax']
        self.hiddenNeurons.append(neurons)
        if activation_function.lower() in allowed:
            self.activation_functions.append(activation_function)
        else:
            print('error_activation function not found')

    def init_params(self):
        self.weights.append(np.random.rand(self.hiddenNeurons[0] , self.n_in) - 0.5)
        self.biases.append(np.random.rand(self.hiddenNeurons[0], 1) - 0.5)
        for n in range(len(self.hiddenNeurons)-1):
            self.weights.append(np.random.rand(self.hiddenNeurons[n+1],self.hiddenNeurons[n]) - 0.5)
            self.biases.append(np.random.rand(self.hiddenNeurons[n+1], 1) - 0.5)
            self.Z.append(0)
            self.dZ.append(0)
            self.A.append(0)
        self.Z.append(0)#offset
        self.dZ.append(0)#offset
        self.A.append(0)#offset
        self.status = True

    def fit(self, x_train, y_train):
        #Must be at least one Hidden layer
        self.A.append(x_train)
        self.y_train = y_train
        m, n = x_train.shape
        self.n_in  = m
        self.samples = n
        self.n_out = self.hiddenNeurons[-1]
        self.init_params()
        self.gradient_descent()

    def evaluate(self, x_test, y_test):
        self.A[0] = x_test
        self.y_train = y_test
        self.forward_propagation()
        predictions = self.get_predictions(self.A[-1])
        accuracy = self.get_accuracy(predictions, y_test)
        print(f'accuracy {(accuracy * 100):.2f} %')

In [343]:
data = pd.read_csv("/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/datasets/mnist_test.csv")

In [344]:
data = np.array(data)
np.random.shuffle(data)
test_set = data[:2000].T
y_test = test_set[0]
x_test = test_set[1:] / 255.0

train_set = data[2000:].T
y_train = train_set[0]
x_train = train_set[1:] / 255.0

In [345]:
nn = NeuralNetwork()
nn.addLayer(neurons=10, activation_function='relu')
nn.addLayer(neurons=10, activation_function='softmax')
nn.fit(x_train, y_train)

Epoch 0, accuracy=11.16%
Epoch 10, accuracy=21.93%
Epoch 20, accuracy=30.06%
Epoch 30, accuracy=36.56%
Epoch 40, accuracy=43.04%
Epoch 50, accuracy=47.85%
Epoch 60, accuracy=52.44%
Epoch 70, accuracy=56.27%
Epoch 80, accuracy=59.75%
Epoch 90, accuracy=62.32%
Epoch 100, accuracy=64.65%
Epoch 110, accuracy=66.90%
Epoch 120, accuracy=69.06%
Epoch 130, accuracy=70.59%
Epoch 140, accuracy=71.95%
Epoch 150, accuracy=73.39%
Epoch 160, accuracy=74.45%
Epoch 170, accuracy=75.58%
Epoch 180, accuracy=76.70%
Epoch 190, accuracy=77.65%
Epoch 200, accuracy=78.31%
Epoch 210, accuracy=78.97%
Epoch 220, accuracy=79.44%
Epoch 230, accuracy=79.99%
Epoch 240, accuracy=80.45%
Epoch 250, accuracy=80.81%
Epoch 260, accuracy=81.27%
Epoch 270, accuracy=81.41%
Epoch 280, accuracy=81.34%
Epoch 290, accuracy=80.86%
Epoch 300, accuracy=78.17%
Epoch 310, accuracy=74.26%
Epoch 320, accuracy=81.79%
Epoch 330, accuracy=74.00%
Epoch 340, accuracy=70.10%
Epoch 350, accuracy=75.58%
Epoch 360, accuracy=75.91%
Epoch 370, a

/var/folders/8n/kyg7fb_d6nxbl39w2mfh9rgh0000gn/T/ipykernel_49396/1356191508.py:33: RuntimeWarning: overflow encountered in exp
  A = np.exp(Z) / sum(np.exp(Z))
/var/folders/8n/kyg7fb_d6nxbl39w2mfh9rgh0000gn/T/ipykernel_49396/1356191508.py:33: RuntimeWarning: invalid value encountered in divide
  A = np.exp(Z) / sum(np.exp(Z))


Epoch 430, accuracy=9.93%
Epoch 440, accuracy=9.93%
Epoch 450, accuracy=9.93%
Epoch 460, accuracy=9.93%
Epoch 470, accuracy=9.93%


KeyboardInterrupt: 